In [ ]:
import pandas, os, sys, re, collections, importlib
sys.path.append('/Users/dp/pma/')
import sameRiver
import sameRiver.biotypeLookupFileMaker
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
import Bio
import pprint
from pprint import pprint as pp
import primerMaker
importlib.reload(primerMaker)
from primerMaker import primerMaker

from Bio import Entrez
from Bio import SeqIO
import time

In [ ]:
from Bio import Entrez
from Bio import SeqIO
import time

def _mk(x):
    if not os.path.exists(x):
        os.system('mkdir ' + x)
_mk('gb/')
_mk('fasta/')

def download_gb(_id="BC040523"):
    Entrez.email = "dfporter@stanford.edu"
    with Entrez.efetch(db="nucleotide", rettype="gb", retmode="text", id=_id) as handle:
        with open('gb/{}.gb'.format(_id), 'w') as f:
            f.write(handle.read())
        handle.close()
        
def download_all_gb(id_list):
    for _id in id_list:
        if type(_id) != type('') or (_id == 'Not found!') or (_id == 'nan'):
            continue
        if os.path.exists('gb/{}.gb'.format(_id)):
            continue
        try:
            download_gb(_id=_id)
        except:
            print("Failed to download {}".format(_id))
        time.sleep(5)
        
        


In [ ]:
def make_enst_gene_name_biotype_map_file(
    gtf_filename='/opt/genome/ensembl_release94_GRCh38/gencode.v29.primary_assembly.annotation.exons_cds_only.gtf',
    out_filename='enst_transcript_id_name_biotype_map.txt'):

    rows = collections.defaultdict(dict)
    
    def re_result(search_result):
        if search_result is None:
            return ''
        else:
            return search_result.group(1)
        
    with open(gtf_filename) as f:
        for li in f:
            #s = li.rstrip('\n').split('\t')
            txpt = re_result(re.search('transcript_id=* *"*([^;"]+)', li))     
            name = re_result(re.search('gene_name=* *"*([^;"]+)', li))

            biotype = re_result(re.search('transcript_biotype=* *"*([^;"]+)', li))
            if (biotype == ''):
                biotype = re_result(re.search('biotype=* *"*([^;"]+)', li))
            if biotype == '':
                biotype = re_result(re.search('transcript_type=* *"*([^;"]+)', li))
            if biotype == '':
                biotype = re_result(re.search('gene_type=* *"*([^;"]+)', li))
            if biotype == '':
                biotype = re_result(re.search('type=* *"*([^;"]+)', li))

            if txpt == '':
                continue
            
            if txpt not in rows:
                rows[txpt] = {'transcript_id': txpt, 'gene_name': name, 'transcript_biotype': biotype,
                             }
    
    with open(out_filename, 'w') as f:
        keys = ['transcript_id', 'gene_name', 'transcript_biotype']
        f.write('\t'.join(keys) + '\n')
        for row in rows.values():
            f.write('\t'.join([row[x] for x in keys]) + '\n')

In [ ]:
amigo = pandas.read_csv('listsOfRbps/rna_binding_genes_from_amigo.txt', sep='\t', comment='#')['Gene name'].tolist()
#print(amigo)
gtf = '/opt/genome/ensembl_release94_GRCh38/gencode.v29.primary_assembly.annotation.exons_cds_only.gtf'


In [ ]:

# Download all the genbank sequences for MGC cDNA clones for RBPs with missense
# mutations in cancer.

def load_rbps(download: bool=False) -> pandas.DataFrame
    df = pandas.read_excel('./RBP_high_freq_mutations.xlsx')

    # Filtering.
    df = df.loc[:, [x for x in df.columns if 'notes' not in x]]
    df['URL'] = df['URL'].astype('str')
    df['URL'] = [x.split('/')[-1] for x in df.URL]
    df['Amigo?'] = [bool(x in amigo) for x in df['Gene']]

    if download:
        download_all_gb(df.URL)
    
    return df

df = load_rbps(download=True)

In [ ]:
# BamHI cut uORF Nt FHH (only see His tag in overlap primer.)

# left_overlap = 'ACATCATCACCACCATCAT' # <- this failed
#left_overlap = 'ACATCATCACCACCATCATGGA'
left_overlap = 'ACATCATCACCACCATCATGGATCC'
#right_overlap = 'CCACACTGGACTAGTGGATC' # <- as in the primer (antisense)
right_overlap = 'GATCCACTAGTCCAGTGTGG' # <- as in the vector (sense)
                #CCACACTGGACTAGTGGATC CTAGTGTCTCTGGGCCAAGG
#        ACATCATCACCACCATCAT   ATGGAATCAAATCACAAATCCGGG
# CELF L ACATCATCACCACCATCATGGAATG AACGGCACCCTGGA
# eIFH L ACATCATCACCACCATCATGGAATG GCGGACTTCGACAC

# R stitch  overlap region   stop  C-term of CDS
# CELF R CCACACTGGACTAGTGGATC TCA GTAGGGCTTGCTGTC
# hnRD R CCACACTGGACTAGTGGATC TTA GTATGGTTTGTAGCTATTTTGATGAC
# eIFH R CCACACTGGACTAGTGGATC TCA TTCTTGCTCCTTTTGAACG
# TIAL R CCACACTGGACTAGTGGATC TCA CTGTGTTTGGTAACTTGC
# gblo.  CCACACTGGACTAGTGGATC TTA GTATGGTTTGTAGCTATTTTGATGACCACCTCGCCT
#        CCACACTGGACTAGTGGATC
#    TCCACCACACTGGACTAGTGGATC
#.       CCACACTGGACTAGTGGATCTTACATCCTACATACAGGTGTTATTCCAA
gblock_start = 'ACATCATCACCACCATCATGGAATGTCGGAGGAGCAGTTCGGAGGTGACGGAGCAGCAGCTGCTGCTAC'

s = Seq(gblock_start[1:])
sp = s.translate()
print(sp)
# SYKPY
gblock_end = 'CAGGCGAGGTGGTCATCAAAATAGCTACAAACCATACTAAGATCCACTAGTCCAGTGTGG'

g = Seq(gblock_end[1+3*12:])
overlap_forward = Seq(gblock_end[37:])
overlap_rc = overlap_forward.reverse_complement()
print(overlap_rc)
print(gblock_end[37:])
g.translate()

_0_rightstitch = 'TGGCAAGCAATAGCGCGGCGGCGAAAGCCGCACGC'
_1_leftstitch = 'ATGGCGTGCGGCTTTCGCCGCCGCGCTATTGCTTG'
print('   ' + Seq(_0_rightstitch).reverse_complement())
print(_1_leftstitch)

from Bio.Data import CodonTable
standard_table = CodonTable.unambiguous_dna_by_id[1]
print(standard_table)
standard_table.back_table['H']

In [ ]:
#https://www.ncbi.nlm.nih.gov/nuccore/BC040523
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC

def get_dna(_loc, seq_record):
    seq = seq_record.seq
    dna =  seq[_loc._start:_loc._end]#, IUPAC.ambiguous_dna)
    if _loc._strand != 1:
        dna = dna.reverse_complement()
    return dna  # Seq object

def find_mutation_to_convert(codon, aa):
    bases = ['A', 'T', 'C', 'G']
    possibilities = {}
#    for nt in [0, 1, 2]:
    for base in bases:
        new_codon = base + codon[1:]
        possibilities[str(base + codon[1:])] = str(new_codon.translate())
    for base in bases:
        new_codon = Seq(codon[0] + base + codon[2], IUPAC.ambiguous_dna)
        possibilities[str(codon[0] + base + codon[2])] = str(new_codon.translate())
    for base in bases:
        new_codon = codon[:2] + base
        possibilities[str(codon[:2] + base)] = str(new_codon.translate())        
    #print(possibilities)
    mutations = [x for x in possibilities if possibilities[x]==aa]
    if len(mutations) < 1:
        print("Could not find an appropriate mutation by SNV. Using an entirely different codon.")
        standard_table = CodonTable.unambiguous_dna_by_id[1]
        return standard_table.back_table[aa]
    else:
        return mutations[0]

def get_cds_from_genbank_record(seq_record):

        # Did we annotate what to PCR?
        highlighted_cds = 0
        starts_with = 0

        for feature in seq_record.features:
            y = feature.__dict__
            
            if 'qualifiers' in y and 'label' in y['qualifiers']:
                
                if y['qualifiers']['label'][0] == 'CDS_to_amplify':
                    highlighted_cds = feature
                    # 0-based.
                    if 'starts_with_aa_num' in y['qualifiers']:
                        starts_with = int(y['qualifiers']['starts_with_aa_num'][0])
                    else:
                        starts_with = 0
        if highlighted_cds:
            cds = highlighted_cds
        # If not, get the first CDS annotated.
        else:
            try:
                cds = [x for x in seq_record.features if x.type=='CDS'][0]
            except:
                print("*" * 14 + "No CDS...")
                return
        
        cds_start = cds.location.start
        cds_dna = seq_record.seq[cds.location.start:cds.location.end]
        if cds.location.strand != 1:
            cds_dna = Seq.reverse_complement(cds_dna)
        cds_aa = cds_dna.translate()

        return cds_dna, cds_aa, starts_with
    
def get_primers(fname, aa_wt, aa_n, aa_mut, gene_name='gene_name'): 
    
    for seq_record in SeqIO.parse(fname, "genbank"):

        print(seq_record.id)
        #print(seq_record.features)

        cds_dna, cds_aa, starts_with = get_cds_from_genbank_record(seq_record)
        
        aa_n -= starts_with
    
        #cds_dna = get_dna(cds.location, seq_record)
        #if 'translation' in cds.qualifiers:
        #    cds_aa = cds.qualifiers['translation'][0]
        #else:
        #    cds_aa = cds.translate(cds_dna)
        print('aa_n = ', aa_n, ' seq length for cds: ', len(cds_dna))
        if aa_n * 3 > len(cds_dna):
            print("*" * 14 + "aa number out of bounds")
            return
        
        snp_left = (aa_n - 1) * 3
        snp_right = aa_n * 3
        codon = cds_dna[snp_left:snp_right]
        context = cds_dna[snp_left-6:snp_right+6].translate()
        print("Observed codon translation:", codon.translate(), ' expected ', aa_wt)
        print("Context: ", context)
        
        if aa_wt != codon.translate():
            print("*" * 14 + "Observed aa did not match expected")
            print("Context: ", cds_dna[snp_left-6:snp_right+6].translate())
            #return
        
        mutation = find_mutation_to_convert(codon, aa_mut)
        
        product_size_full = len(cds_dna)
        product_size_left = len(cds_dna[:snp_right])
        product_size_right = len(cds_dna[snp_left:])
        
        if os.path.exists('outputs/pcr_product_sizes.txt'):
            f = open('outputs/pcr_product_sizes.txt', 'a')
        else:
            f = open('outputs/pcr_product_sizes.txt', 'w')
            f.write("seqid\tfull_cds_length\tleft\tright\t-\tCDS\tmut\n")

        f.write(f"{gene_name}\t{product_size_full}\t{product_size_left}")
        f.write(f'\t{product_size_right}')
        f.write("\t-\t{}\t{}\n".format(product_size_full/3 - 1, f"{aa_wt}{aa_n}{aa_mut}"))
            
        print("Mutation {} > {}".format(codon, mutation))
        
        # Amplification primers for whole cds into
        # BamHI cut Nt FHH pLEX uORF.
        pm = primerMaker(blocks=[str(cds_dna)], left=left_overlap, right=right_overlap)
        pm.pick_primers()
        
        with open('fasta/{}.fa'.format(gene_name), 'w') as f:
            f.write(pm.get_seq_as_fasta())
            
        lines = pm.one_per_line()
        
        lines = '\n'.join([x for x in lines.split('\n') if 'stit' in x])
        bc_acc = re.sub('BC0', '', seq_record.id)
        lines = re.sub('_0_', '', lines)
        lines = re.sub('leftstitch', 'L_{}'.format(gene_name), lines)
        lines = re.sub('rightstitch', 'R_{}'.format(gene_name), lines)
        #print(pm.__dict__)
        
        # SDM primers for amplification in two parts for
        # Infusion stitching.
        mut_cds_dna = cds_dna[:snp_left] + mutation + cds_dna[snp_right:]

        pm = primerMaker(blocks=[
            str(mut_cds_dna[:snp_left]), str(mut_cds_dna[snp_left:])],
            left=left_overlap, right=right_overlap)
        
        pm.pick_primers()
        
        with open('fasta/{}_{}.fa'.format(gene_name, str(aa_n) + aa_mut), 'w') as f:
            f.write(pm.get_seq_as_fasta())
            
        lines_sdm = pm.one_per_line().split('\n')
        lines_sdm = '\n'.join([x for x in lines_sdm if 'stitch' in x])
        lines_sdm = re.sub('_0', '{}_0'.format(str(aa_n) + aa_mut), lines_sdm)
        lines_sdm = re.sub('_1', '{}_1'.format(str(aa_n) + aa_mut), lines_sdm)
        lines_sdm = re.sub('_leftstitch', 'L_{}'.format(gene_name), lines_sdm)
        lines_sdm = re.sub('_rightstitch', 'R_{}'.format(gene_name), lines_sdm)
        
        #print(lines_sdm)
        return lines + '\n' + lines_sdm + '\n'
    
import glob, re
def parse_mutation(_str, return_no_mutation=False):
    a = re.split('(\d+)', _str)
    if return_no_mutation:
        return a[0], int(a[1]), a[0]
    return a[0], int(a[1]), a[2]

order = ''
genbank_files = glob.glob('gb/*.gb') + glob.glob('gb/*.gbk')

df = load_rbps()
def get_fname(gene_name):
    
    if type(gene_name) != type(''):
        return False
    
    matching_gbs = [x for x in genbank_files if gene_name in str(x)]

    if len(matching_gbs):
        fname = matching_gbs[0]
    else:
        fname = 'gb/{}.gb'.format(df.iloc[n]['URL'])
        
    if not os.path.exists(fname):
        print("No file {}".format(fname))
        return False
    
    return fname

# Read all CDSes from genbank files and write to a single fasta
# in case this is handy later for BLASTing against expected 
# constructs.
all_cds = ''
cds = {}
for n in range(df.shape[0]):
    
    gene_name = df.iloc[n]['Gene']
    fname = get_fname(gene_name)

    if fname:
        for seq_record in SeqIO.parse(fname, "genbank"):
            cds_dna, cds_aa, starts_with = get_cds_from_genbank_record(seq_record)
            all_cds += f">{gene_name}_{seq_record.id}_{fname}\n"
            all_cds += f"{cds_aa}\n"
            cds[gene_name] = cds_aa
            
with open('outputs/all_expected_cds_from_genbank_input_files.fa', 'w') as f:
    f.write(all_cds)
    
print('Exported all CDSes from genbank files.')

# Clear the file outputs/pcr_product_sizes.txt that is used
# for checking PCR lengths and mutations by hand.
if os.path.exists('outputs/pcr_product_sizes.txt'):
    os.system('rm outputs/pcr_product_sizes.txt')

# Get primers for each mutation.
for n in range(df.shape[0]):
    print('---=---' * 7)
    
    gene_name = df.iloc[n]['Gene']
    
    fname = get_fname(gene_name)

    print(gene_name, fname)
    
    if fname:
        mut = parse_mutation(
            df.iloc[n]['Mutation'])#, return_no_mutation=1)
        #if gene_name in cds:
        #    midpoint = int(len(cds[gene_name])/2)
        #    mut = cds[gene_name][midpoint], midpoint + 1, cds[gene_name][midpoint]
        #else:
        #    print(f"No CDS {gene_name}")
        lines = get_primers(fname, *mut, gene_name=df.iloc[n]['Gene'])
        if lines:
            order += lines

#Write a text order sheet.
#Well Position	Name	Sequence
#A1	test1	agagagaagagagagag
with open('order.txt', 'w') as f:
    f.write(order)

# Write excel order sheet.
rows = []
alphabet = 'ABCDEFGHIJKLMNOPQRST'
letter_index = 0
number_index = 1
for li in order.split('\n'):
    
    # For now, skip the odd man out.
    #if 'RARS2' in li:
    #    continue
    #if li.count('_') < 2:
    #    continue
    if 'RARS2' in li or ('_0L_' not in li and '_1R_' not in li):
        if len(li.split('\t')) < 2:
            continue
        
        rows.append({
            'Well Position': alphabet[letter_index]+str(number_index),
            'Name': li.split('\t')[0],
            'Sequence': li.split('\t')[1].rstrip('\n'),
            'Gene name': li.split('\t')[0].split('_')[-1],
        })
        
        if number_index >= 8:
            letter_index += 1
            number_index = 1
        else:
            number_index += 1
        if letter_index > 7:
            letter_index = 0
            number_index = 1
            
order_df = pandas.DataFrame(rows)
order_df.to_excel('order.xlsx', )

print(order_df)

In [ ]:
def how_many_rbps_do_we_have():
    all_rbps = set()
    all_mutations = {}
    for fname in ['./primerPlates//plate1 IDT primer order 053119 RBP missense mutations TCGA all cancer.xlsx',
                 './primerPlates/plate2 IDT primer order 053119 RBP missense mutations TCGA all cancer.xlsx',
                 './primerPlates/Plate 3 IDT order.xlsx']:
        df = pandas.read_excel(fname)
        genes = pandas.read_excel(fname)['Gene name'].tolist()
        genes = set([x for x in genes if type(x) == type('')])
        count = {rbp:len(df.loc[df['Gene name']==rbp].index) for rbp in genes}
        count = {rbp:(x-2)/2 for rbp, x in count.items()}
        all_mutations.update(count)
        print(count)
        print(genes)
        print("{}: {} cDNAs".format(fname, len(genes)))
        all_rbps |= genes
    print("Total RBPs: {}\n{}".format(len(all_rbps), all_rbps))
    total_mutations = sum(all_mutations.values())
    print("Total mutations: {}\n{}".format(total_mutations, all_mutations))
    total_new_clip_datasets = \
        len(all_rbps) * 2 + ( # WT replicates
        total_mutations - len(all_rbps)) * 2  # Mutant replicates
    
    print("Number of CLIP datasets if 2 replicates/RBP: {}".format(total_new_clip_datasets))
    print("...Added to current datasets (+50) = {}".format(50 + total_new_clip_datasets))
how_many_rbps_do_we_have()

In [ ]:
a = 'AA AAA AAA AAA AAA AAA AAA AAA A'
a.count('A')

In [ ]:
listOfExps = glob.glob('/Users/dfporter/pma/dataAndScripts/clip/experiments/*/*xlsx')
for fname in listOfExps:
    try:
        xl = pandas.ExcelFile(fname)
        bca = [x for x in xl.sheet_names if re.search('bca', x, re.IGNORECASE)]
    except:
        #print(f"Error reading {fname}")
        continue

    for sheet in bca:
        df = xl.parse(sheet)
        asStr = df.to_csv( path_or_buf=None)
        for li in asStr.split('\n'):
            if re.search('HCT116', li, re.IGNORECASE):
                if '60' in li:
                    print(fname, '\n', li)

In [ ]:
_str = """190703_elim_internal_cds_sequencing_of_minipreps_rbp_missense			
1	"6/23 NpL uORF DICER1 A3"	DICER1_aa200F	
2	"6/23 NpL uORF DICER1 A3"	DICER1_aa375F	
3	"6/23 NpL uORF DICER1 A3"	DICER1_aa600F	
4	"6/23 NpL uORF DICER1 A3"	DICER1_aa800F	
5	"6/23 NpL uORF DICER1 A3"	DICER1_aa1000F	
6	"6/23 NpL uORF DICER1 A3"	DICER1_aa1200F	
7	"6/23 NpL uORF DICER1 A3"	DICER1_aa1400F	
8	"6/23 NpL uORF DICER1 A3"	DICER1_aa1600F	
9	"7/2 DICER R944Q T2-9 NpL"	DICER1_aa200F	
10	"7/2 DICER R944Q T2-9 NpL"	DICER1_aa375F	
11	"7/2 DICER R944Q T2-9 NpL"	DICER1_aa600F	
12	"7/2 DICER R944Q T2-9 NpL"	DICER1_aa800F	
13	"7/2 DICER R944Q T2-9 NpL"	DICER1_aa1000F	
14	"7/2 DICER R944Q T2-9 NpL"	DICER1_aa1200F	
15	"7/2 DICER R944Q T2-9 NpL"	DICER1_aa1400F	
16	"7/2 DICER R944Q T2-9 NpL"	DICER1_aa1600F	
17	"7/2 RBM11 R74C T3-1 NpL"	DICER1_aa200F	
18	"7/2 RBM11 R74C T3-1 NpL"	DICER1_aa375F	
19	"7/2 RBM11 R74C T3-1 NpL"	DICER1_aa600F	
20	"7/2 RBM11 R74C T3-1 NpL"	DICER1_aa800F	
21	"7/2 RBM11 R74C T3-1 NpL"	DICER1_aa1000F	
22	"7/2 RBM11 R74C T3-1 NpL"	DICER1_aa1200F	
23	"7/2 RBM11 R74C T3-1 NpL"	DICER1_aa1400F	
24	"7/2 RBM11 R74C T3-1 NpL"	DICER1_aa1600F	
25	B5 A1CF	A1CF_aa200F	
26	B5 A1CF	A1CF_aa500R	
27	"7/2 mini KPNB1 552F T1-1 NpL"	KPNB1_aa200F	
28	"7/2 mini KPNB1 552F T1-1 NpL"	KPNB1_aa400F	
29	"7/2 mini KPNB1 552F T1-1 NpL"	KPNB1_aa850R	
30	"KPNB1 6/26 NpL uORF"	KPNB1_aa200F	
31	"KPNB1 6/26 NpL uORF"	KPNB1_aa400F	
32	"KPNB1 6/26 NpL uORF"	KPNB1_aa850R	
33	"DDX50 6/26 NpL ini 1 uORF"	DDX50_aa200F	
34	"DDX50 6/26 NpL ini 1 uORF"	DDX50_aa400F	
35	DDX3X 528C E6	DDX3X_aa200F	
36	DDX3X 528C E6	DDX3X_aa400F	
37	DDX3X 528C E11	DDX3X_aa200F	
38	DDX3X 528C E11	DDX3X_aa400F	
39	"7/2 DICER R944Q T2-9 NpL"	pTRE5	
40	"7/2 RBM11 R74C T3-1 NpL"	pTRE5	
41	"7/2 Rbfox1 69T T3-6 NpL"	pTRE5	
42	"72 A1CF E34K T4-3 NpL"	pTRE5	"""
#a = pandas.read_clipboard(sep='\t')
#print(a)
#df = a
df.columns = ['rxn',
             'name', 'primer', 'NA']
print(df.head())

from Bio import SeqIO

from Bio.SeqRecord import SeqRecord
def reformat_sanger(fasta_fname, _df):
    df = _df.copy()
    records = []
    for seq_record in SeqIO.parse(fasta_fname, "fasta"):

        num = int(seq_record.id.split('-')[-1].split('_')[0]) - 1
#        print(df.loc[int(num), 'name'])
        new_id = df.loc[num, 'name'] + ', ' + df.loc[num, 'primer'] + ', ' + seq_record.id

        seq_record.id = re.sub(' ', '_', new_id)
        records.append(SeqRecord(seq_record.seq, id=seq_record.id))
        name = re.sub(',', '_', seq_record.id)
        name = re.sub('/', '', name)
        SeqIO.write(seq_record, os.path.dirname(fasta_fname) + f'/edited_names/{name}.fa', 'fasta') 

    SeqIO.write(records, os.path.dirname(fasta_fname) + '/all_edited_names.fa',
               'fasta')
reformat_sanger(
    '/Users/dfporter/Desktop/sanger/seqs/190703_elim_miniprep_sequencing_internal_cds_primers/all.fa',
    df)



In [ ]:
fname = '/Users/dfporter/Downloads/HWUDSRJE114-Alignment.txt'

outli = ''
in_aligns = False
with open(fname) as f:
    for li in f:
        if in_aligns and re.search('\AQuery= ', li):
            in_aligns = False
        elif in_aligns:
            continue
        elif re.search('\AALIGNMENTS', li):
            in_aligns = True
            continue

        outli += li

with open(os.path.dirname(fname) + '/edited.txt', 'w') as f:
    f.write(outli)

In [ ]:

print(len('dyeeedeeeeSLMWRAPKEEADYEDDF'))

In [ ]:
import pprint as pp
def how_many_rbps_are_verified(fname):
    df = pandas.read_excel(fname, sheet_name='By protein')
    results = collections.defaultdict(set)

    for row in df.to_dict('records'):
        
        construct = str(row['Unnamed: 1'])
        if construct in ['nan', '<-']:
            continue
            
        verif = any([re.search('Verif', str(x), re.IGNORECASE) for x in row.values()])
        if verif:
            results['Have constructs'].add(construct)
            results['Have RBPs'].add(construct.split(' ')[0])
        else:
            results['Missing RBPs'].add(construct.split(' ')[0])
        
        results['Missing RBPs'] -= results['Have RBPs']
        
        n_constructs = {
            name: len([x for x in results['Have constructs'] if name in x]) for name in results['Have RBPs']}
        results['at_least_two'] = set([
            x for x, v in n_constructs.items() if v>1])
        results['only_one_construct'] = set([
            x for x, v in n_constructs.items() if v<=1])
    for k, v in results.items():
        print(f'{k}: {len(v)}')    
    pp.pprint(results)
    
how_many_rbps_are_verified('./Sequencing_schemes_and_results.xlsx')